# Distribución Metros Cuadrados Construidos


In [1]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    if nombre.startswith("df_") and "puebla" in nombre    
    and ("alfa_q" in nombre or "jul_2023" in nombre or "sep_2023" in nombre or "feb_2024" in nombre or "mar_2024" in nombre or "may_2024" in nombre)
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_alfa_q_puebla',
 'df_feb_2024_puebla',
 'df_jul_2023_puebla',
 'df_mar_2024_puebla',
 'df_may_2024_puebla',
 'df_sep_2023_puebla']

In [2]:
# Iterar sobre cada DataFrame en la lista filtrada
for nombre_df in nombres_df_filtrados:
    # Obtener el DataFrame usando globals()
    df = globals()[nombre_df]
    
    df.rename(columns={'id_jul_23':'id','m2_contruido': 'm2_construido','Metros_construido':'m2_construido','segmento':'categoria','Category':'categoria'}, inplace=True)
        # Asignar el DataFrame modificado de nuevo a la variable global
    globals()[nombre_df] = df 

# Imprimir confirmación
print("Columnas renombradas en los DataFrames filtrados.")


Columnas renombradas en los DataFrames filtrados.


In [3]:
df_alfa_q_puebla.columns

Index(['id', 'Q', 'categoria', 'ids', 'Propiedad', 'Precio', 'Metros_total',
       'm2_construido', 'Precio_M2_Terreno', 'Precio_M2_Construido',
       'Personas_interesadas', 'Tiempo_publicacion',
       'Promedio_Interes_Mensual', 'Estado', 'Tipo', 'Estacionamientos',
       'Recamaras', 'Banos', 'Medio_banos', 'Banos_Total', 'Cantidad_de_pisos',
       'Antiguedad', 'Seguridad', 'Fraccionamiento', 'Colonia', 'CP', 'url'],
      dtype='object')

In [4]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
dataframes_name = []

for nombre_df in nombres_df_filtrados:
    # Seleccionar las columnas 'id' y 'categoria'
    segment_df = globals()[nombre_df][['id', 'categoria','m2_construido']]
    # Añadir el DataFrame a la lista
    dataframes_list.append(segment_df)
     # Componer el nombre del DataFrame en la lista dataframes_name
    dataframes_name.append(nombre_df)


# Cantidad de registros por metros cuadrados construidos, por segmento

In [5]:
def procesar_dataframes(dataframes_list, dataframes_name):
    for i, df in enumerate(dataframes_list):
        # Crear una nueva columna con la primera letra de la categoría
        df['segmento'] = df['categoria'].str[0]
        df.drop(columns=['categoria'], inplace=True)

        # Clasificar 'baño_total' en rangos específicos
        bins = [0, 100, 150, 200, 250, 300, float('inf')]
        labels = ['0-100', '100-150', '150-200', '200-250', '250-300', '300+']
        df['m2_construido'] = pd.cut(df['m2_construido'], bins=bins, labels=labels, right=False)

        # Agrupar y contar los datos
        tabla = df.groupby(['segmento', 'm2_construido']).size().unstack(fill_value=0)

        # Imprimir el nombre del DataFrame y la tabla resultante
        print(f"Nombre del DataFrame: {dataframes_name[i]}")
        print(tabla)

# Procesar los DataFrames de la lista
procesar_dataframes(dataframes_list, dataframes_name)

Nombre del DataFrame: df_alfa_q_puebla
m2_construido  0-100  100-150  150-200  200-250  250-300  300+
segmento                                                      
A                  5       20       12        4        2     0
B                 18       16        5        0        0     0
C                 11       17        1        0        0     0
D                  7        0        0        0        0     0
S                  2       17        5        9        4    31
Nombre del DataFrame: df_feb_2024_puebla
m2_construido  0-100  100-150  150-200  200-250  250-300  300+
segmento                                                      
A                 34      134      126      172       37    45
B                 51      233      206       63       30    32
C                178      243       76       40       18    37
D                307      188       59       42        7    22
E                237      135      121      180       88   223
L                  0        0        0

C:\Users\yoe11\AppData\Local\Temp\ipykernel_14904\275059014.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['segmento'] = df['categoria'].str[0]
C:\Users\yoe11\AppData\Local\Temp\ipykernel_14904\275059014.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['categoria'], inplace=True)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_14904\275059014.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:

## Tabla general de frecuencia
 Recuento total de filas para cada combinación de valores únicos de 'segmento' y 'm2_contruido' con todos los DataFrame

In [6]:
# Crear una nueva función para sumar las tablas de todos los DataFrames
def sumar_tablas(dataframes_list):
    # Inicializar una tabla vacía para almacenar la suma
    tabla_suma = None
    
    # Iterar sobre cada DataFrame en la lista
    for df in dataframes_list:
        # Agrupar y contar los datos en el DataFrame actual
        tabla_actual = df.groupby(['segmento', 'm2_construido']).size().unstack(fill_value=0)
        
        # Sumar la tabla actual al acumulador (tabla_suma)
        if tabla_suma is None:
            tabla_suma = tabla_actual
        else:
            tabla_suma = tabla_suma.add(tabla_actual, fill_value=0)
    
    # Imprimir la tabla resultante de la suma
    print("Tabla general de frecuencia:")
    print(tabla_suma)

# Llamar a la función sumar_tablas con la lista de DataFrames como argumento
sumar_tablas(dataframes_list)

Tabla general de frecuencia:
m2_construido   0-100  100-150  150-200  200-250  250-300    300+
segmento                                                         
A               254.0    566.0    513.0    663.0    192.0   208.0
B               342.0    895.0    774.0    313.0     98.0   172.0
C               803.0   1027.0    345.0    130.0     70.0    96.0
D              1317.0    685.0    169.0    114.0     38.0    70.0
E               853.0    445.0    349.0    489.0    230.0   643.0
L                 0.0      0.0      1.0      5.0      4.0   640.0
S                83.0    315.0    423.0    556.0    501.0  2037.0


C:\Users\yoe11\AppData\Local\Temp\ipykernel_14904\470711682.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  tabla_actual = df.groupby(['segmento', 'm2_construido']).size().unstack(fill_value=0)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_14904\470711682.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  tabla_actual = df.groupby(['segmento', 'm2_construido']).size().unstack(fill_value=0)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_14904\470711682.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain curren

In [7]:
def sumar_tablas(dataframes_list):
    tabla_suma = None
    
    for df in dataframes_list:
        tabla_actual = df.groupby(['m2_construido', 'segmento']).size().unstack(fill_value=0)
        
        if tabla_suma is None:
            tabla_suma = tabla_actual
        else:
            tabla_suma = tabla_suma.add(tabla_actual, fill_value=0)
    
    print("Tabla general de frecuencia:")
    print(tabla_suma)
    
    fig = graficar_tabla(tabla_suma)
    
    # Guardar la gráfica como archivo HTML
    guardar_grafico_como_html(fig, 'g_bar_dist_m2_construidos', carpeta='assets/graficas')
    
    return fig

colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6','#4225CF','#2CBA4D']

def graficar_tabla(tabla):  
    segmentos = tabla.index
    banos = tabla.columns
    
    fig = go.Figure()
    
    for i, segmento in enumerate(segmentos):
        color = colores[i % len(colores)]
        fig.add_trace(go.Bar(
            x=banos,
            y=tabla.loc[segmento],
            name=segmento,
            marker_color=color
        ))
    
    fig.update_layout(
        xaxis_title="Segmento",        
        yaxis=dict(
            title='Número de casas',
            gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y
            gridwidth=1,
        ),
        legend_title="Rango de M2",
        barmode='group',
        legend=dict(
            orientation='h',  # Orientación horizontal de la leyenda
            yanchor='bottom',  # Anclar al borde inferior
            y=1.02,  # Colocar la leyenda justo debajo del gráfico
            xanchor='right',
            x=1
        ),
        plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
        margin=dict(l=10, r=10, t=10, b=10) # Ajusta los márgenes (left, right, top, bottom)
    )    
    
    fig.show()    
    return fig

def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Guardar la gráfica como archivo HTML en la carpeta especificada
    ruta_archivo = os.path.join(carpeta, f'{nombre_archivo}.html')
    pio.write_html(fig, ruta_archivo)
# Llamar a la función sumar_tablas con la lista de DataFrames como argumento
fig = sumar_tablas(dataframes_list)

C:\Users\yoe11\AppData\Local\Temp\ipykernel_14904\3114147233.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  tabla_actual = df.groupby(['m2_construido', 'segmento']).size().unstack(fill_value=0)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_14904\3114147233.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  tabla_actual = df.groupby(['m2_construido', 'segmento']).size().unstack(fill_value=0)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_14904\3114147233.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain cur

Tabla general de frecuencia:
segmento         A    B     C     D      E      L     S
m2_construido                                          
0-100          254  342   803  1317  853.0    0.0    83
100-150        566  895  1027   685  445.0    0.0   315
150-200        513  774   345   169  349.0    1.0   423
200-250        663  313   130   114  489.0    5.0   556
250-300        192   98    70    38  230.0    4.0   501
300+           208  172    96    70  643.0  640.0  2037


In [8]:
def sumar_tablas(dataframes_list):
    tabla_suma = None
    
    for df in dataframes_list:
        tabla_actual = df.groupby(['m2_construido', 'segmento']).size().unstack(fill_value=0)
        
        if tabla_suma is None:
            tabla_suma = tabla_actual
        else:
            tabla_suma = tabla_suma.add(tabla_actual, fill_value=0)
    
    print("Tabla general de frecuencia:")
    print(tabla_suma)
    
    fig = graficar_tabla(tabla_suma)
    
    # Guardar la gráfica como archivo HTML
    guardar_grafico_como_html(fig, 'g_pie_dist_m2_construidos', carpeta='assets/graficas')
    
    return fig

colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6','#4225CF','#2CBA4D']

def graficar_tabla(tabla):
    # Sumar los valores por 'm2_contruido' para obtener el total por rango de M2
    suma_por_m2 = tabla.sum(axis=1)
    
    fig = go.Figure(data=[go.Pie(
        labels=suma_por_m2.index.astype(str),
        values=suma_por_m2.values,
        marker=dict(colors=colores),
        textinfo='label+percent',
        insidetextorientation='radial'
    )])
    
    fig.update_layout(
        # title="Distribución de M2 construidos",
        legend_title="Rango de M2",
        barmode='group',
        legend=dict(
            orientation='h',  # Orientación horizontal de la leyenda
            yanchor='bottom',  # Anclar al borde inferior
            y=1.02, 
            xanchor='right',
            x=0.72
        ),
        plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
        margin=dict(l=10, r=10, t=10, b=10) # Ajusta los márgenes (left, right, top, bottom)
    )    
    
    fig.show()
    return fig

def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Guardar la gráfica como archivo HTML en la carpeta especificada
    ruta_archivo = os.path.join(carpeta, f'{nombre_archivo}.html')
    pio.write_html(fig, ruta_archivo)
fig1 = sumar_tablas(dataframes_list)

Tabla general de frecuencia:
segmento         A    B     C     D      E      L     S
m2_construido                                          
0-100          254  342   803  1317  853.0    0.0    83
100-150        566  895  1027   685  445.0    0.0   315
150-200        513  774   345   169  349.0    1.0   423
200-250        663  313   130   114  489.0    5.0   556
250-300        192   98    70    38  230.0    4.0   501
300+           208  172    96    70  643.0  640.0  2037


C:\Users\yoe11\AppData\Local\Temp\ipykernel_14904\1287722546.py:5: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\yoe11\AppData\Local\Temp\ipykernel_14904\1287722546.py:5: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\yoe11\AppData\Local\Temp\ipykernel_14904\1287722546.py:5: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\yoe11\AppData\Local\Temp\ipykernel_14904\1287722546.py:5: FutureWarning:

Th